In [1]:
import copy
import json
import math
import fiona
import folium
import branca.colormap as cm
import requests
import numpy as np
import pandas as pd
import geopandas as gp
from shapely.geometry import Polygon
import urllib.request
from urllib import request
from shapely.geometry import shape, Point
import os
import threading
from rtree import index
import time
import datetime

In [2]:
#========================================================================================================
#爬蟲 (環保署測站、時間、風力資訊、台中各區天氣) + idw

ses = requests.Session()
data1 = ses.get('http://taqm.epb.taichung.gov.tw/TQAMNEWAQITABLE.ASPX') #環保署16筆測站
data1.encoding = 'utf-8'
t = pd.read_html(data1.text)[0]
t.drop(t.iloc[:, 1:21], inplace=True, axis=0)
t.drop(t.iloc[:, 1:279], inplace=True, axis=1)
times = str(t[0])



In [3]:
#爬蟲 微型感測器

url = 'https://aqi.thu.edu.tw/echarts/getEPAIotData'
json_data = request.urlopen(url).read().decode("utf-8")
json_data = json.loads(json_data);
frame = pd.DataFrame.from_dict(json_data);

frame['areaname']=''
frame.index= range(1,len(frame) + 1)
frame['Id']=frame.index
frame.index= range(0,len(frame))
times_micro = frame['datetime']
frame

,datetime,device_id,latitude,longitude,pm25,areaname,Id
0,2019-12-21 01:10:37,1858224692,24.975056,121.330402,7,,1
1,2019-12-21 01:35:35,1858300612,24.956991,121.345565,-99,,2
2,2019-12-21 01:37:52,1858495000,25.042832,121.137675,-99,,3
3,2019-12-21 01:38:08,1858569336,24.967298,121.339279,9,,4
4,2019-12-21 01:35:44,1858763627,24.973724,121.332481,18,,5
5,2019-12-21 01:37:42,1858852981,24.972902,121.342242,8,,6
6,2019-12-20 14:36:39,1858981737,24.976159,121.334666,7,,7
7,2019-12-20 08:04:43,1859886331,24.965728,121.332249,8,,8
8,2019-12-21 01:38:05,1860067717,24.94805,121.341753,7,,9
9,2019-12-21 01:37:58,1860271203,24.956835,121.360893,7,,10


In [4]:
#找出位於台中的微型感測器
#先找出微型感測器所在位置
def getareaname(lng,lat,num):
   
    lng = float(lng)
    lat = float(lat)
    
    area_append['areaname'][num]=search(float(lng), float(lat))
    return 0

def get_areanamebythread(totaldata):
    totalnum=totaldata  #總執行次數
    Q=int(totalnum/5) #取商數
    R=totalnum%5     #取餘數
    
    for i in range(Q):
        threads = []
        for j in range(5):#開多少執行緒
            threads.append(threading.Thread(
                target=getareaname,#要執行函數
                args=(frame['longitude'][i*5+j],frame['latitude'][i*5+j],i*5+j)))#要執行函數的參數
            threads[j].start()
        for j in threads:
            j.join()
        print(round(float((i+1)*100/totalnum*5),2),'%')#顯示進度
    
    threads = []
    for i in range(R):
        threads.append(threading.Thread(
            target=getareaname,
            args=(frame['longitude'][Q*5+i],frame['latitude'][Q*5+i],Q*5+i)))
        threads[i].start()
    for j in threads:
        j.join()
    print("100%")#顯示進度

def search(x, y):  #尋找鄉鎮
    global shapes, townnames
    #idx = index.Index()
    #for town_id, shape in shapes.items():
    #idx.insert(town_id,shape.bounds)
    return next((townnames[town_id]  #如果鄉鎮區域包含傳入的經緯度就傳回townnames[town_id]
                 for town_id in shapes #idx.intersection((x,y)) #逐一尋找各鄉鎮
                 if shapes[town_id].contains(Point(x, y))), None)

area_append = pd.DataFrame(columns=['areaname'])
area_append['areaname'] = frame['datetime']



module_dir = os.path.dirname('/home/gh555657/123321/areasearch/')  #取得目前目錄
collection = fiona.open(os.path.join(module_dir,'TOWN_MOI_1070516.shp'))
shapes = {}
townnames = {}

for f in collection:
    town_id =f['properties']['TOWNCODE'] #鄉鎮代碼
    shapes[town_id] = shape(f['geometry'])  #鄉鎮界限經緯度
    townnames[town_id] = f['properties']['COUNTYNAME'] + f['properties']['TOWNNAME']#search函式傳回值
    
t1=time.time()
get_areanamebythread(frame.shape[0])
t2=time.time()
print('gevent-time:%s' % str(t2-t1))
frame['areaname']=area_append['areaname']


0.18 %
0.35 %
0.53 %
0.7 %
0.88 %
1.06 %
1.23 %
1.41 %
1.58 %
1.76 %
1.94 %
2.11 %
2.29 %
2.46 %
2.64 %
2.82 %
2.99 %
3.17 %
3.35 %
3.52 %
3.7 %
3.87 %
4.05 %
4.23 %
4.4 %
4.58 %
4.75 %
4.93 %
5.11 %
5.28 %
5.46 %
5.63 %
5.81 %
5.99 %
6.16 %
6.34 %
6.51 %
6.69 %
6.87 %
7.04 %
7.22 %
7.39 %
7.57 %
7.75 %
7.92 %
8.1 %
8.27 %
8.45 %
8.63 %
8.8 %
8.98 %
9.15 %
9.33 %
9.51 %
9.68 %
9.86 %
10.04 %
10.21 %
10.39 %
10.56 %
10.74 %
10.92 %
11.09 %
11.27 %
11.44 %
11.62 %
11.8 %
11.97 %
12.15 %
12.32 %
12.5 %
12.68 %
12.85 %
13.03 %
13.2 %
13.38 %
13.56 %
13.73 %
13.91 %
14.08 %
14.26 %
14.44 %
14.61 %
14.79 %
14.96 %
15.14 %
15.32 %
15.49 %
15.67 %
15.85 %
16.02 %
16.2 %
16.37 %
16.55 %
16.73 %
16.9 %
17.08 %
17.25 %
17.43 %
17.61 %
17.78 %
17.96 %
18.13 %
18.31 %
18.49 %
18.66 %
18.84 %
19.01 %
19.19 %
19.37 %
19.54 %
19.72 %
19.89 %
20.07 %
20.25 %
20.42 %
20.6 %
20.77 %
20.95 %
21.13 %
21.3 %
21.48 %
21.65 %
21.83 %
22.01 %
22.18 %
22.36 %
22.54 %
22.71 %
22.89 %
23.06 %
23.24 %
23.42 %
23.5

In [5]:
frame['pm25'].replace({"NA":np.nan}, inplace=True)
frame['pm25'].astype('float64')
frame['pm25'].replace({"-99":np.nan,"-99.0":np.nan}, inplace=True)
frame[frame['pm25']<0]=np.nan
frame.dropna(inplace=True)
frame

,datetime,device_id,latitude,longitude,pm25,areaname,Id
0,2019-12-21 01:10:37,1858224692,24.975056,121.330402,7.0,新北市鶯歌區,1.0
3,2019-12-21 01:38:08,1858569336,24.967298,121.339279,9.0,新北市鶯歌區,4.0
4,2019-12-21 01:35:44,1858763627,24.973724,121.332481,18.0,新北市鶯歌區,5.0
5,2019-12-21 01:37:42,1858852981,24.972902,121.342242,8.0,新北市鶯歌區,6.0
6,2019-12-20 14:36:39,1858981737,24.976159,121.334666,7.0,新北市鶯歌區,7.0
7,2019-12-20 08:04:43,1859886331,24.965728,121.332249,8.0,新北市鶯歌區,8.0
8,2019-12-21 01:38:05,1860067717,24.94805,121.341753,7.0,新北市鶯歌區,9.0
9,2019-12-21 01:37:58,1860271203,24.956835,121.360893,7.0,新北市鶯歌區,10.0
10,2019-12-21 01:37:55,1860464577,24.973736,121.328505,19.0,新北市鶯歌區,11.0
11,2019-12-21 01:38:22,1860726455,24.95694,121.358481,7.0,新北市鶯歌區,12.0


In [6]:
now_time=(datetime.datetime.now()+datetime.timedelta(hours=0)).strftime("%Y-%m-%d %H")
frame=frame.loc[frame["datetime"].str.contains(now_time)]
frame.to_csv('/home/gh555657/123321/taiwan_all_micro.csv')

In [7]:
frame

,datetime,device_id,latitude,longitude,pm25,areaname,Id
0,2019-12-21 01:10:37,1858224692,24.975056,121.330402,7.0,新北市鶯歌區,1.0
3,2019-12-21 01:38:08,1858569336,24.967298,121.339279,9.0,新北市鶯歌區,4.0
4,2019-12-21 01:35:44,1858763627,24.973724,121.332481,18.0,新北市鶯歌區,5.0
5,2019-12-21 01:37:42,1858852981,24.972902,121.342242,8.0,新北市鶯歌區,6.0
8,2019-12-21 01:38:05,1860067717,24.94805,121.341753,7.0,新北市鶯歌區,9.0
9,2019-12-21 01:37:58,1860271203,24.956835,121.360893,7.0,新北市鶯歌區,10.0
10,2019-12-21 01:37:55,1860464577,24.973736,121.328505,19.0,新北市鶯歌區,11.0
11,2019-12-21 01:38:22,1860726455,24.95694,121.358481,7.0,新北市鶯歌區,12.0
13,2019-12-21 01:19:37,1861018038,24.950129,121.348118,8.0,新北市鶯歌區,14.0
15,2019-12-21 01:38:03,1862163982,24.956971,121.353984,7.0,新北市鶯歌區,16.0


In [8]:
epa = frame.loc[frame["areaname"].str.contains('台中|臺中')]
epa=epa.drop(columns=['datetime','areaname']).reset_index(drop=True)
epa.columns=['SiteName','Latitude','Longitude','PM2.5','Id']
epa.index= range(1,len(epa) + 1)
epa['Id']=epa.index
epa.index= range(0,len(epa))
epaidw=epa
epa=epa[['SiteName','Id','PM2.5','Latitude','Longitude']]
epaidw=epaidw.drop(columns=['SiteName']).reset_index(drop=True)
epa

,SiteName,Id,PM2.5,Latitude,Longitude
0,6170369574,1,10.0,24.161848,120.6049575
1,6170479504,2,4.0,24.237501,120.517601
2,6170511673,3,4.0,24.241976,120.523674
3,6170644194,4,9.0,24.15854,120.6066032
4,6170777118,5,6.0,24.166212,120.5971678
5,6170852684,6,7.0,24.206862,120.5050963
6,6170988978,7,3.0,24.234808,120.5295139
7,6171173697,8,4.0,24.236878,120.5057026
8,6171278726,9,5.0,24.242703,120.5328221
9,6171387123,10,8.0,24.303089,120.7221755


In [9]:
epaidw.columns=['Latitude','Longitude','PM2.5','Id']

In [10]:

epaidw=epaidw[['Id','Latitude','Longitude','PM2.5']]



In [11]:
# print(times[21:36]) 時間
df1 = pd.read_html(data1.text)[1]
cols1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
df1 = df1.replace('─','0')
df1 = df1.drop(df1.columns[cols1],axis=1)
df1.rename(columns={ df1.columns[0]: "SiteName"}, inplace=True)
df1.rename(columns={ df1.columns[1]: "PM2.5"}, inplace=True)
df1['Latitude']=[24.1622,24.151958,24.099611,24.225628,
                 24.256586,24.139008,24.350426,24.139564,
                 24.05735,24.094264,24.307036,24.250388,
                 24.150919,24.182055,24.269233,24.20103]

df1['Longitude']=[120.616917,120.641092,120.677689,120.568794,
                  120.741711,120.597876,120.615358,120.715064,
                  120.697299,120.646629,120.714881,120.538839,
                  120.540877,120.60707,120.576421,120.498566]


In [12]:
df1.rename(columns={"測站名稱":"SiteName"}, inplace=True)
df1.rename(columns={ "細懸浮微粒(PM2.5)":"PM2.5"}, inplace=True)
df1

,SiteName,PM2.5,Latitude,Longitude
,SiteName,移動平均濃度(μg/m3),,
0,西屯測站(環保署),12,24.162200,120.616917
1,忠明測站(環保署),13,24.151958,120.641092
2,大里測站(環保署),13,24.099611,120.677689
3,沙鹿測站(環保署),9,24.225628,120.568794
4,豐原測站(環保署),10,24.256586,120.741711
5,文山測站,19,24.139008,120.597876
6,大甲測站,9,24.350426,120.615358
7,太平測站,14,24.139564,120.715064
8,霧峰測站,13,24.057350,120.697299


In [13]:
df1[['Latitude', 'Longitude', 'PM2.5']] = df1[['Latitude', 'Longitude','PM2.5']].astype(float)
#df1  顯示16筆測站
df1.to_csv("/home/gh555657/123321/df1.csv")
df1 = pd.read_csv("/home/gh555657/123321/df1.csv")

In [14]:
df1.drop([0],inplace=True) #把df1不要的第一row砍掉
df11=df1['SiteName']       #先測站把名稱存進df11
#df11=df11.append(epa['SiteName'])#在把微型感測器測站名稱也併進df11

df1.drop(["SiteName"],axis=1,inplace=True)#測站名稱column砍掉
df1.rename(columns={ 'Unnamed: 0':'Id'}, inplace=True)#更改unname--->Id
#df1=df1[['Id','SiteName','PM2.5','Latitude','Longitude']] df1目前型態
#df1=df1.append(epaidw) #把微型感測器資料也併進來
df1=df1.astype('float64')#轉成float64
df11.index= range(0,len(df11))#index重排
df1.index=range(0,len(df1))

In [15]:
#taichung = gp.read_file("/home/hpc/taichungcity.geojson")           #台中邊界
taichungmap_1x1 = gp.read_file("/home/gh555657/123321/final.geojson")         #台中1*1網格
taichung_district = gp.read_file("/home/gh555657/123321/taichung_district.geojson")
#list1= [   1,    4,   14,   26,   44,   63,   82,  102,  122,  144,
#         168,  193,  221,  257,  304,  353,  403,  455,  510,  568,
#         627,  687,  750,  819,  892,  968, 1053, 1141, 1232, 1325,
#        1418, 1510, 1601, 1692, 1781, 1864, 1944, 2019, 2087, 2145,
#        2197, 2246, 2289, 2330, 2359, 2384, 2403, 2419, 2433, 2445 ]
#list2= [   3,   13,   25,   43,   62,   81,  101,  121,  143,  167,
#         192,  220,  256,  303,  352,  402,  454,  509,  567,  626,
#         686,  749,  818,  891,  967, 1052, 1140, 1231, 1324, 1417,
#        1509, 1600, 1691, 1780, 1863, 1943, 2018, 2086, 2144, 2196,
#        2245, 2288, 2329, 2358, 2383, 2402, 2418, 2432, 2444, 2449 ]
lon_max=taichungmap_1x1.bounds.maxx
lon_min=taichungmap_1x1.bounds.minx
lat_max=taichungmap_1x1.bounds.maxy
lat_min=taichungmap_1x1.bounds.miny

In [16]:
print(df1.shape)
df1

(16, 4)


,Id,PM2.5,Latitude,Longitude
0,0.0,12.0,24.162200,120.616917
1,1.0,13.0,24.151958,120.641092
2,2.0,13.0,24.099611,120.677689
3,3.0,9.0,24.225628,120.568794
4,4.0,10.0,24.256586,120.741711
5,5.0,19.0,24.139008,120.597876
6,6.0,9.0,24.350426,120.615358
7,7.0,14.0,24.139564,120.715064
8,8.0,13.0,24.057350,120.697299
9,9.0,16.0,24.094264,120.646629


In [17]:
#lat_max,lat_min,lon_max,lon_min四份合併做成DataFrame
df_grid = pd.DataFrame([lat_max,lat_min,lon_max,lon_min]).T
df_grid['grid_Longitude'] = (df_grid['maxx']+df_grid['minx'])/2
df_grid['grid_Latitude']  = (df_grid['maxy']+df_grid['miny'])/2
df_grid['grid_ID']   = df_grid.index+1
df_grid

,maxy,miny,maxx,minx,grid_Longitude,grid_Latitude,grid_ID
0,24.004430,23.995383,120.744002,120.734155,120.739079,23.999907,1
1,24.004446,23.995400,120.753831,120.743984,120.748907,23.999923,2
2,24.004461,23.995416,120.763659,120.753813,120.758736,23.999939,3
3,24.013349,24.004299,120.685010,120.675158,120.680084,24.008824,4
4,24.013369,24.004320,120.694839,120.684988,120.689914,24.008844,5
5,24.013388,24.004340,120.704668,120.694818,120.699743,24.008864,6
6,24.013407,24.004359,120.714497,120.704648,120.709572,24.008883,7
7,24.013425,24.004378,120.724326,120.714477,120.719402,24.008901,8
8,24.013442,24.004396,120.734155,120.724307,120.729231,24.008919,9
9,24.013459,24.004413,120.743984,120.734137,120.739061,24.008936,10


In [18]:
#定義IDW

def idw(data_obs,data_grid,radius,interval):
    print('半徑 %.2Fkm 範圍內無觀測站則無法觀測'%(radius+interval))#*2
    df_goal = pd.DataFrame([])
    for grid_i in range(len(data_grid)):
        data_obs['grid_ID'] = data_grid.loc[grid_i,'grid_ID']
        df_work = pd.merge(data_grid,data_obs,how='inner',on='grid_ID')
    
        size= 0.0092
    
        df_work['Lat_dis_km']   = abs((df_work['Latitude']  - df_work['grid_Latitude']) /size)
        df_work['Lon_dis_km']   = abs((df_work['Longitude'] - df_work['grid_Longitude'])/size)
        df_work['distance']     = np.sqrt(df_work['Lat_dis_km']**2+df_work['Lon_dis_km']**2)
        df_work['distance_rec'] = 1/df_work['distance']

        df_work = df_work.sort_values(by=['distance'])
        df_work = df_work.reset_index()

 
        #做出圓心
        center_list = df_work.loc[df_work['distance']<=radius]
        df_center = pd.DataFrame([])
        if len(center_list)==0:
            df_center = pd.DataFrame({'grid_ID':[grid_i+1],'center_PM2.5':[None]})
        else:
            center_temp = center_list.groupby('grid_ID')['distance_rec'].sum()
            center_list = pd.merge(center_list,center_temp,how='inner',on='grid_ID')
            center_list['center_PM2.5']=(center_list['distance_rec_x']/center_list['distance_rec_y'])*center_list['PM2.5']
            df_center = pd.DataFrame(center_list.groupby(['grid_ID'])['center_PM2.5'].sum()).reset_index()  
    
        #做出第1層圓環
        L1_list = df_work.loc[(df_work['distance']>radius)&(df_work['distance']<=(radius+interval))]
        df_L1 = pd.DataFrame([])
        if len(L1_list)==0:
            df_L1 = pd.DataFrame({'grid_ID':[grid_i+1],'L1_PM2.5':[None]})
        else:
            L1_temp = L1_list.groupby('grid_ID')['distance_rec'].sum()
            L1_list = pd.merge(L1_list,L1_temp,how='inner',on='grid_ID')
            L1_list['L1_PM2.5']=(L1_list['distance_rec_x']/L1_list['distance_rec_y'])*L1_list['PM2.5']
            df_L1 = pd.DataFrame(L1_list.groupby(['grid_ID'])['L1_PM2.5'].sum()).reset_index()
 
        #做出第2層圓環
        L2_list = df_work.loc[(df_work['distance']>(radius+interval))&(df_work['distance']<=(radius+interval*2))]
        df_L2 = pd.DataFrame([])
        if len(L2_list)==0:
             df_L2 = pd.DataFrame({'grid_ID':[grid_i+1],'L2_PM2.5':[None]})
        else:
            L2_temp = L2_list.groupby('grid_ID')['distance_rec'].sum()
            L2_list = pd.merge(L2_list,L2_temp,how='inner',on='grid_ID')
            L2_list['L2_PM2.5']=(L2_list['distance_rec_x']/L2_list['distance_rec_y'])*L2_list['PM2.5']
            df_L2 = pd.DataFrame(L2_list.groupby(['grid_ID'])['L2_PM2.5'].sum()).reset_index()

    
        #合併數據(join)
        df_all = pd.merge(pd.merge(df_center,df_L1,on='grid_ID'),df_L2,on='grid_ID')
        df_all['radius'] = radius
        df_all['interval'] = interval

        #總合併(union)
        df_goal = pd.concat([df_goal,df_all],sort=True)
    return df_goal

In [19]:
#執行idw()並deepcopy切割
df_goal = copy.deepcopy(idw(df1,df_grid,5,10))

半徑 15.00km 範圍內無觀測站則無法觀測


In [20]:
df_goal.reset_index(inplace=True)

In [21]:
#製作狀況D的圓心推估值(center_adj)
df_goal['dis_weight'] = ((df_goal['interval']*1.5+df_goal['radius'])/(df_goal['interval']*0.5+df_goal['radius']))-1
df_goal['adj'] = (df_goal['L2_PM2.5']-df_goal['L1_PM2.5'])
df_goal['center_adj']   =  df_goal['L1_PM2.5']-df_goal['adj']*df_goal['dis_weight']

In [22]:
df_goal.head()

,index,L1_PM2.5,L2_PM2.5,center_PM2.5,grid_ID,interval,radius,dis_weight,adj,center_adj
0,0,13.752263,13.706289,NaN,1,10,5,1.0,-0.045974,13.798237
1,0,13.000000,14.816091,NaN,2,10,5,1.0,1.816091,11.183909
2,0,13.000000,14.804141,NaN,3,10,5,1.0,1.804141,11.195859
3,0,13.822746,13.233967,NaN,4,10,5,1.0,-0.588779,14.411525
4,0,13.788558,13.222324,NaN,5,10,5,1.0,-0.566234,14.354792


In [23]:
'''
依照狀況A~D，給上不同的最終推估值(est_PM2.5)
共有以下A~D四種狀況：
A.半徑5km內至少有1個觀測站，則估計值直接使用center_PM2.5。
B.半徑5km內沒有任何觀測站、第一層圓環至少有1觀測站、第二層圓環沒有任何觀測站，則估計值直接使用L1_PM2.5。
C.半徑5km內沒有任何觀測站、第一層圓環沒有任何觀測站、第二層圓環至少有1觀測站，則估計值直接使用L2_PM2.5或無法估計。
D.半徑5km內沒有任何觀測站、第一層圓環至少有1觀測站、第二層圓環至少有1觀測站，則使用L1_PM2.5和L2_PM2.5的遞減遞增估計。
E.第二圓環以內完全沒有任何觀測站，即半徑25km以內完全沒有觀測站，則不應該估計數值：無法估計。
'''
condition_A =  ~df_goal['center_PM2.5'].isnull()
condition_B = ( df_goal['center_PM2.5'].isnull())&(~df_goal['L1_PM2.5'].isnull())&( df_goal['L2_PM2.5'].isnull())
condition_C = ( df_goal['center_PM2.5'].isnull())&( df_goal['L1_PM2.5'].isnull())&(~df_goal['L2_PM2.5'].isnull())
condition_D = ( df_goal['center_PM2.5'].isnull())&(~df_goal['L1_PM2.5'].isnull())&(~df_goal['L2_PM2.5'].isnull())
        
df_goal['est_PM2.5'] = df_goal['center_PM2.5']
df_goal['est_PM2.5'].loc[condition_A]=df_goal['center_PM2.5'].loc[condition_A]
df_goal['est_PM2.5'].loc[condition_B]=df_goal['L1_PM2.5'].loc[condition_B]
df_goal['est_PM2.5'].loc[condition_C]=-1
#df_goal['est_PM2.5'].loc[condition_C]=df_goal['L2_PM2.5'].loc[condition_C]
df_goal['est_PM2.5'].loc[condition_D]=df_goal['center_adj'].loc[condition_D]


/home/gh555657/.local/lib/python3.5/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [24]:
#產出df3以供後續應用
df3 = pd.merge(df_goal,df_grid,how='inner',on='grid_ID')
df3 = df3[['grid_Latitude','grid_Longitude','est_PM2.5','grid_ID']]
df3.columns = ['Latitude','Longitude','PM2.5','Id']
df3.loc[df3['PM2.5'].isnull(),'PM2.5']=-1
df3

,Latitude,Longitude,PM2.5,Id
0,23.999907,120.739079,13.798237,1
1,23.999923,120.748907,11.183909,2
2,23.999939,120.758736,11.195859,3
3,24.008824,120.680084,14.411525,4
4,24.008844,120.689914,14.354792,5
5,24.008864,120.699743,14.321716,6
6,24.008883,120.709572,14.314202,7
7,24.008901,120.719402,14.328034,8
8,24.008919,120.729231,13.923241,9
9,24.008936,120.739061,13.960605,10


In [25]:
df3.to_csv("/home/gh555657/123321/all_point_data_epa.csv")
df3.to_csv("/home/gh555657/123321/testidw2_only16.csv")
all_point_data_epa = pd.read_csv("/home/gh555657/123321/all_point_data_epa.csv")
lon=list(all_point_data_epa['Longitude'])
lat=list(all_point_data_epa['Latitude'])
#all_point_data_epa['Id']=0
Id=taichungmap_1x1['Id']
ans_Id=all_point_data_epa['Id']

In [26]:
def generalID(lon,lat,column_num,row_num):
    # 若在范围外的点，返回-1
    if lon <= LON1 or lon >= LON2 or lat <= LAT1 or lat >= LAT2:
        return -1
    # 把经度范围根据列数等分切割
    column = (LON2 - LON1)/column_num
    # 把纬度范围根据行数数等分切割
    row = (LAT2 - LAT1)/row_num
    # 二维矩阵坐标索引转换为一维ID，即： （列坐标区域（向下取整）+ 1） + （行坐标区域 * 列数）
    return int((lon-LON1)/column)+ 1 + int((lat-LAT1)/row) * column_num


In [27]:
# fit id
#for i in range(2449):
#    ans_Id[i] = i+1
# for d in (range(100)):      #測試資料筆數
#      for m in (range(50)):  #找是在哪列
#             if lon_min[list1[m]-1]<lon[d] and lon[d]<lon_max[list2[m]-1] and lat_min[list1[m]-1]<lat[d] and lat[d]<lat_max[list2[m]-1]:
#                 for n in range(list1[m],list2[m]+1): #找是在該列的哪個
#                     if lon_min[n-1]<lon[d] and lon[d]<lon_max[n-1] and lat_min[n-1]<lat[d] and lat[d]<lat_max[n-1]:
#                         ans_Id[d]=Id[n]-1

#taichung_district

In [28]:
taichungmap_1x1 = taichungmap_1x1.merge(all_point_data_epa, on='Id')
taichungmap_1x1['PM2.5']=taichungmap_1x1['PM2.5'].round()
#taichung_district = taichung_district.merge(weather, on='區名')


In [29]:
df1['SiteName']=df11
df1=df1[['Id','SiteName','PM2.5','Latitude','Longitude']]


In [30]:
# =============================================================================================================
# folium
#(215, 207, 207, 0.00)
variable = 'PM2.5'
colorList = [(215, 207, 207, 1), '#98fb98', '#51ff51', '#00ff00', '#1ce11c',
             '#32cd32', '#ffff00', '#ffee00', '#ffd13f', '#ffc700', '#ffbf4a',
             '#ffa500', '#ff6347', '#ff5047', '#ff4c2c', '#ff0000', '#d32c4a',
             '#ba55d3']
map_color = cm.StepColormap(colorList,index=[-1,0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
    vmin=-1,vmax=100,caption = 'PM2.5')

fmap = folium.Map(location=[24.2, 120.9], zoom_start=10.5)
# fmap.choropleth(
#                geo_data=taichungmap_1x1,
#                name='pm2.5',
#                columns=['Id', 'PM2.5'],
#                key_on='feature.properties.Id',
#                data=all_point_data_epa,
#               #threshold_scale=[],
#                fill_color='BuGn',
#                legend_name='pm2.5',
#                line_opacity=0.5,
#                fill_opacity=0.8
#                )
folium.GeoJson(taichungmap_1x1,
               name='PM2.5',
               style_function=lambda x: {
                   'fillColor': map_color(x['properties'][variable]),
                   'color': 'black',
                   'weight': 0,
                   'fillOpacity': 0.7
               },
               highlight_function=lambda x: {
                   'weight': 3,
                   'color': 'black'
               },
               tooltip=folium.GeoJsonTooltip(fields=['Id', 'PM2.5'],
                                             aliases=['Id', 'PM2.5'],
                                             labels=True,
                                             sticky=True)).add_to(fmap)


#微型感測器logo
epa_micro_url= 'https://ci.taiwan.gov.tw/dsp/img/map_icon/air_quality.png'
# 環保署 logo
epa_icon_url = 'https://www.epa.gov.tw/public/MMO/epa/Epa_Logo_01_LOGO.gif'


station = folium.FeatureGroup(name="環保署", show=True)
for i in (range(15)):
    station.add_child(
        folium.Marker(
            location=[df1['Latitude'][i], df1['Longitude'][i]],
            popup=("<b>NAME:</b> {NAME}<br>"
                   " <p><b>PM2.5:</b> {PM25}<br>"
                   " <p><b>TIME:</b> {TIME}<br>").format(
                       NAME=str(df1['SiteName'][i]),
                       PM25=str(df1['PM2.5'][i]),
                       TIME=str(times[39:55])),
            icon=folium.CustomIcon(epa_icon_url,
                                   icon_size=(23,
                                              23))  # Creating a custom Icon
        ))


#fmap.add_child(epa_micro)
fmap.add_child(station)

fmap.add_child(map_color)
folium.LayerControl().add_to(fmap)
# lat/lon to map
# folium.LatLngPopup().add_to(fmap)
fmap.save('/var/www/html/pm25_only16.html')  # 存成 final.html

In [31]:
 times_micro

0       2019-12-21 01:10:37
1       2019-12-21 01:35:35
2       2019-12-21 01:37:52
3       2019-12-21 01:38:08
4       2019-12-21 01:35:44
5       2019-12-21 01:37:42
6       2019-12-20 14:36:39
7       2019-12-20 08:04:43
8       2019-12-21 01:38:05
9       2019-12-21 01:37:58
10      2019-12-21 01:37:55
11      2019-12-21 01:38:22
12      2019-12-21 01:31:38
13      2019-12-21 01:19:37
14      2019-12-21 01:37:37
15      2019-12-21 01:38:03
16      2019-12-21 01:37:06
17      2019-12-21 01:37:41
18      2019-12-21 01:37:43
19      2019-12-21 01:35:06
20      2019-12-21 01:37:40
21      2019-12-21 01:38:03
22      2019-12-21 01:38:13
23      2019-12-21 00:51:40
24      2019-12-21 01:38:00
25      2019-12-21 01:37:12
26      2019-12-20 17:01:47
27      2019-12-20 21:06:35
28      2019-12-20 20:10:33
29      2019-12-21 01:35:34
               ...         
2810    2019-12-21 01:34:58
2811    2019-12-21 01:34:56
2812    2019-12-21 01:38:12
2813    2019-12-21 01:35:24
2814    2019-12-21 0